# Sieci konwolucyjne w Tensorflow dla realnych zbiorów danych

### Pobierzmy dane (zbiór realnych zdjęć kotów i psów z kaggle'a, pobierany przez stronę Microsoftu)

In [2]:
%pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.0-cp310-cp310-macosx_11_0_arm64.whl (120 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests

url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
r = requests.get(url, allow_redirects=True)
open('dogs-vs-cats.zip', 'wb').write(r.content)

824887076

### Rozpakujmy skompresowany folder

In [5]:
!unzip -qq dogs-vs-cats.zip

### Stwórzmy trzy podzbiory danych: treningowy, testowy i walidacyjny

In [6]:
import os, shutil, pathlib
import imghdr
original_dir = pathlib.Path("PetImages")
new_base_dir = pathlib.Path("cats_vs_dogs")
img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]

def make_subset(subset_name, start_index, end_index):
    for category in ("Cat", "Dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{10000+i}.jpg" for i in range(start_index, end_index)]

        for fname in fnames:
          img_type = imghdr.what(original_dir/category/fname)
          if img_type in img_type_accepted_by_tf:
            shutil.copyfile(src=original_dir / category / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1300)
make_subset("test", start_index=1500, end_index=2500)

### Użyjmy klasy `image_dataset_from_directory` do wygenerowania struktur które posłużą jako zbiory ustandaryzowanych danych dla sieci

**Zadanie 3a. Patrząc na definicję zbioru treningowego, stwórz zbiór testowy i walidacyjny**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
img_shape = (img_size[0] , img_size[1] , 3)
train_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()
valid_gen = ImageDataGenerator()

train_dataset = train_gen.flow_from_directory(
    directory = f"{new_base_dir}/train", 
    target_size = img_size,
    class_mode = 'binary',
    color_mode = 'rgb',
    shuffle = True ,
    batch_size = 32)
validation_dataset = ...
test_dataset = ...

### Spójrzmy na rozmiar i wymiary zbiorów danych przed paczkowaniem i po paczkowaniu

In [ ]:
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

In [ ]:
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

In [ ]:
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break

In [ ]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

### Zbudujmy nieduży model konwolucyjny do klasyfikacji zdjęć w Tensorflow

**W bibiliotece Tensorflow możemy dodawać kolejne warstwy sieci w postaci funkcji layer(x) gdzie x to poprzednia warstwa a layer to obecna warstwa.**

**Zadanie 3b. Dokończ architekturę modelu, dodając na końcu kolejno:**
- warstwę konwolucyjną z 2 razy większą liczbą filtrów niż wcześniejsza, takim samym rozmiar kernela i taką samą fukcją aktywacji, 
- warstwę poolingową identyczną jak wcześniejsze
- warstwę konwolucyjną identyczną jak poprzednia
- warstwę spłaszczającą (`Flatten`)
- wyjściową warstwę gęstą o jednym neuronie, która zwraca `wartość prawdpodobieństwa` - warstwę tę przypisz do zmiennej `outputs`

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(224, 224, 3))                               # warstwa wejściowa
x = layers.Rescaling(1./255)(inputs)                                    # warstwa normalizująca
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)      # warstwa konwolucyjna
x = layers.MaxPooling2D(pool_size=2)(x)                                 # warstwa poolingowa
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
...



outputs = 
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 178, 178, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 89, 89, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 87, 87, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 43, 43, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 41, 41, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 18, 18, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 9, 9, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 7, 7, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │          12,545 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 991,041 (3.78 MB)

 Trainable params: 991,041 (3.78 MB)

 Non-trainable params: 0 (0.00 B)

**Zadanie 4a. Przygotuj model do treningu, z `binarną krosentropią` jako funkcją straty, `rmsprop` jako optymalizatorem i `trafnością` jako metryką.**

**Zadanie 4b. Naucz model na zbiorze treningowym w 15 epokach, z walidacją na zbiorze walidacyjnym i z użyciem callbacka zapisującego najlepszy model i przypisz historię uczenia do zmiennej `history`**

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath="convnet.keras", save_best_only=True, monitor="val_loss")]
history = ...

### Zobaczmy performance sieci

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

### Dokonajmy ewaluacji modelu

In [ ]:
test_model = keras.models.load_model("convnet.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

### Wykonajmy poszerzenie zbioru danych (`data augmentation`)

**Zdefiniujmy pierwszy fragment sieci, który będzie odpowiadał za modyfikację próbek**

In [49]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

### Zobaczmy zmodyfikowane w ten sposób, losowe próbki 

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(15,15))
for i in range(4):
  # convert to unsigned integers for plotting
  image = next(train_dataset)
  image = data_augmentation(image)
  # changing size from (1, 200, 200, 3) to (200, 200, 3) for plotting the image
  image = np.squeeze(image)
  # plot raw pixel data
  ax[i].imshow(image[0].astype("uint8"))
  ax[i].axis('off')

### Stwórzmy nowy model, który uwzględni regularyzację poprzez rozszerzenie zbioru danych

**Zadanie 5a. Stwórz nowy model sieci konwolucyjnej, analogiczny do poprzedniego, ale ze zdefiniowanym powyżej fragmentem `'augmentacyjnym'` zaraz za wejściem oraz z warstwą `dropout`, o parametrze np. 0.5 (sprawdź w dokumentacji), między warstwą spłaszczającą a warstwą wyjściową**

**Zadanie 5b. Skompiluj model, naucz (historię przypisz do zmiennej `history`) i dokonaj jego ewaluacji**

In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath="convnet_augmentation.keras", save_best_only=True, monitor="val_loss")]
history = ...

In [ ]:
test_model = keras.models.load_model(
    "convnet_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

### Ponownie prześledźmy proces uczenia

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

**Zadanie 5c. Jakie dostrzegasz różnice w procesie uczenia z bazowym i poszerzonym zbiorem danych?**

tu odpowiedź